<a href="https://colab.research.google.com/github/TeddLe1804/TeddLe1804/blob/main/%08%5B4_2023%5DBreast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ENVIRONMENT SETUP 

In [ ]:
# ADd yolox 
!git clone git@github.com:Megvii-BaseDetection/YOLOX.git
cd YOLOX
!pip3 install -v -e .  # or  python3 setup.py develop

In [ ]:
# https://stackoverflow.com/questions/46288847/how-to-suppress-pip-upgrade-warning
!pip config set global.disable-pip-version-check true
!pip config set global.root-user-action ignore

Writing to /root/.config/pip/pip.conf
Writing to /root/.config/pip/pip.conf


In [ ]:
import os

In [ ]:
os.environ['CUDA_MODULE_LOADING'] = 'LAZY'
#This line of code is used to reduce the memory used of importing module if only the module is needed 

In [ ]:
!mkdir -p /kaggle/tmp/libs

In [ ]:
!pip install --index-url https://test.pypi.org/simple/ --no-deps torch2trt

Looking in indexes: https://test.pypi.org/simple/, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.4 MB/s eta 0:00:00


In [ ]:
!pip install tensorrt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.2/819.2 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.6/719.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 379.5/379.5 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.5/823.5 kB 71.2 MB/s eta 0:00:00


In [ ]:
import torch2trt

In [ ]:
import sys

import cv2
import numpy as np
import torch
import torchvision
from torch2trt import TRTModule
from torch.nn import functional as F

In [ ]:
_TORCH_VER = [int(x) for x in torch.__version__.split(".")[:2]]
_TORCH11X = (_TORCH_VER >= [1, 10])

## EXTRACT ROI DETECTION 

In [ ]:
def meshgrid(*tensors):
    if _TORCH11X:
        return torch.meshgrid(*tensors, indexing="ij")
    else:
        return torch.meshgrid(*tensors)


In [ ]:
def extract_roi(img, kernel=(5,5)): 
  ori_h, ori_w = img.shape[:2]

  # Clip percentage: implant, white lines 
  upper = np.percentile(img, 95)
  img[img > upper] = np.min(img)

  # Gaussian filtering to reduce noise 
  if kernel is not None: 
    img = cv2.GaussianBlur(img, kernel, 0)
    _, img_bin = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

  #Dilation to improve the coutours connectivity using OpenCV Morphological Operations
  element = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3), (-1, -1))
  img_bin = cv2.dilate(img_bin, element)
  contours, _ = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

  if len(contours) == 0: 
    return None, None, None 

  areas = np.array([cv2.contourArea(contour) for contour in contours])
  select_idx = np.argmax(areas) 
  contour = contours(select_idx)
  area_pct = areas[select_idx] / (img.shape[0] * img.shape[1])
  x0, y0, w, h = cv2.boundingRect(contour)

  #Min-max safety in roi 
  x1 = min(max(int(x0 + w), 0), ori_w)
  y1 = min(max(int(y0 + h), 0), ori_h)
  x0 = min(max(int(x0), 0), ori_w)
  y0 = min(max(int(y0), 0), ori_h)
  return [x0, y0, x1, y1], area_pct, None


In [ ]:
class RoiExtractor: 
  def __init__(self,
               engine_path, 
               input_size, 
               num_classes, 
               conf_thres = 0.5, 
               nms_thres = 0.9,
               class_agnostic = False, 
               area_pct_thres = 0.04, 
               hw=None, 
               strides=None, 
               exp=None): 
               
               self.input_size, self.input_h, self.input_w = input_size
               self.num_classes = num_classes
               self.conf_thres = conf_thres
               self.nms_thres = nms_thres 
               self.class_agnostic = class_agnostic 
               self.area_pct_thres = area_pct_thres 

                  # Call Nvidia GPU 
               model = TRTModule()
               model.load_state_dict(torch.load(engine_path))
               self.model = model
               if hw in None or strides is None: 
                 assert exp is not None 
                 self._set_meta(exp)
               else: 
                self.hw = hw 
                self.strides = strides
  def _set_meta(self, exp): 
    assert exp is not None
    print("Start probing model metadata")

    torch_model = exp.get_model().cuda().eval()
    _dummy = torch.ones(1,3, exp.test_size[0], exp.test_size[1])
    torch_model(_dummy)
    
    #Set attribute
    self.hw = torch_model.head.hw
    self.strides = torch_model.head.strides 

    #Clean up 
    del torch_model, _dummy
    import gc 
    gc.collect()
    torch.cuda.empty_cache()
    print('Done probbing model metadata')


    def decode_output(self, outputs):
      dtype = outputs.type()
      grids = []
      strides = []
      for (hszie, wsize), stride in zip(self.hw, self.strides): 
        yv, xv = meshgrid([torch.arange(hsize), torch.arange(wsize)])
        grid = torch.stack((xv, yv), 2).view(1,-1,2)
        grids.append(grid)
        shape = grid.shape[:2]
        strides.append(torch.full((*shape, 1), stride))
      
      grids = torch.cat(grids, dim=1).type(dtype)
      strides = torch.cat(strides, dim=1).type(dtype)

      output = torch.cat(
          [(outputs[..., 0:2] + grids) * strides,
           torch.exxp(outputs[..., 2:4]) * strides, outputs[..., 4:]],
           dim = -1)
      return outputs 


      def post_process(self,
                       pred, 
                       conf_thres=0.5,
                       nms_thres=0.9,
                       class_agnostic=False):
        '''
        This code block takes in a tensor pred with dimensions [batch_size, num_boxes, num_coords] 
        and generates the coordinates of the corners of the bounding boxes represented by the pred tensor.

First, a new tensor box_corner is created of the same shape as pred. 
The first dimension of box_corner (i.e. batch_size) is the same as that of pred. 
The second dimension of box_corner (i.e. num_boxes) is also the same as that of pred. 
The third dimension of box_corner (i.e. num_coords) is set to 4, 
as this tensor stores the coordinates of the top left and bottom right corners of the bounding boxes.

The four coordinates of a bounding box are calculated as follows:

box_corner[:, :, 0] = pred[:, :, 0] - pred[:, :, 2] / 2

This computes the x-coordinate of the top left corner of each bounding box, 
which is equal to the x-coordinate of the center of the box (pred[:, :, 0])
 minus half of the width of the box (pred[:, :, 2] / 2).

box_corner[:, :, 1] = pred[:, :, 1] - pred[:, :, 3] / 2

This computes the y-coordinate of the top left corner of each bounding box, 
which is equal to the y-coordinate of the center of the box (pred[:, :, 1]) 
minus half of the height of the box (pred[:, :, 3] / 2).

box_corner[:, :, 2] = pred[:, :, 0] + pred[:, :, 2] / 2

This computes the x-coordinate of the bottom right corner of each bounding box,
 which is equal to the x-coordinate of the center of the box (pred[:, :, 0]) plus half of the width of the box (pred[:, :, 2] / 2).

box_corner[:, :, 3] = pred[:, :, 1] + pred[:, :, 3] / 2

This computes the y-coordinate of the bottom right corner of each bounding box, which is equal to the y-coordinate of the center of the box (pred[:, :, 1]) plus half of the height of the box (pred[:, :, 3] / 2).

Finally, the first four coordinates of the pred tensor are updated with the corresponding coordinates from box_corner tensor. This effectively updates the coordinates of the bounding boxes from center coordinates to corner coordinates.
        '''
        box_corner = pred.new(pred.shape)
        box_corner[:, :, 0] = pred[:, :, 0] - pred[:, :, 2] / 2 
        box_corner[:, :, 1] = pred[:, :, 1] - pred[:, :, 3] / 2
        box_corner[:, :, 2] = pred[:, :, 0] + pred[:, :, 2] / 2 
        box_corner[:, :, 3] = pred[:, :, 1] + pred[:, :, 3] / 2 
        pred[:, :, :4] = box_corner[:, :, :4]

        output = [None for _ in range(len(pred))]
        for i, image_pred in enumerate(pred): 

          # If none are remaining => process next image 
          if not image_pred.size(0): 
            continue 

          # Get score and class with highest confidence 
          class_conf, class_pred = torch.max(image_pred[:, 5:5 + self.num_classes], 1, keepdim=True) 

          #Conf mask 
          conf_mask = (image_pred[:, 4] * class_conf.squeeze() >= conf_thres).squeeze()

          #Detections ordered as (x1, y1, x2, y2, obj_conf, class_conf, class_pred)
          detections = torch.cat(
              (image_pred[:, :5], class_conf, class_pred.float()), 1
          )

          detections = detections[conf_mask]
          
          if not detections.size(0): 
            continue 
          
          if class_agnostic: 
            nms_out_index = torchvision.ops.nms(
                detections[:, :4],
                detections[:, 4] * detections[:, 5], 
                nms_thres,
            )
          else:
            nms_out_index = torchvision.ops.batch_nms(
                detections[:, :4],
                detections[:, 4] * detections[:, 5], 
                detections[:, 6], 
                nms_thres,
            )

            detections = detections[nms_out_index]
            if output[i] is None: 
              output[i] = detections 
            else:
              output[i] = torch.cat((output[i], detections))
        return output 


        # Preprocess the single one 
        def preprocess_single(self, img: torch.Tensor): 
          ori_h = img.size[0]
          ori_w = img.size[1]

          ratio = min(self.input_h / ori_h, self.input / ori_w)
          '''
          This ratio is necessary because we want to ensure that the input image does not exceed the 
          maximum  allowed size for the model, and to preserve the original aspect ratio of the image.
          '''

          resized_img = F.interpolate(img.view(1,1, ori_h, ori_w),
                                      mode = 'bilinear',
                                      scale_factor = ratio,
                                      recompute_scale_factor=True)[0,0]
          
          # Padding 
          padded_img = torch.full((self.input_h, self.input_w),
                                  114,
                                  dtype = resized_img.dtype,
                                  device ='cuda') #Sẽ thay đổi thông số này --> TÌm thông số tối ưu

          padded_img[:resized_img.size(0), :resized_img.size(1)] = resized_img 
          padded_img = padded_img.unsqueeze(-1).expand(-1, -1, 3) 
          # The "dim -1" parameter specifies that this new dimension should be added at the end of the tensor. This function is useful when working with multi-dimensional arrays that require an additional dimension for processing or operations.
          #In PyTorch, unsqueeze() is a commonly used function that allows you to increase the number of dimensions in a tensor. For instance, you can use unsqueeze() to convert a 1D tensor into a 2D tensor by adding a new dimension. This can be used in a wide range of applications in machine learning, 
          #such as image processing, natural language processing, and more.
          #Overall, unsqueeze() with dim -1 is a powerful tool for manipulating and processing multi-modal data in Python-based machine learning frameworks like PyTorch.

          padded_img = padded_img.permuate(2, 0, 1) #Đảo chiều tensor 
          # [0, 1 ,2] --> Permute(2,0,1) --> [2, 1, 0]
          padded_img = padded_img.float()
          return padded_img, resized_img, ratio, ori_h, ori_w 
        
        def detect_single(self, img): 
          


